# MLOps using only standard Python

## Configuration

In [17]:
data_directory = 'data'
init_data_url = 'https://github.com/wodecki/datasets/blob/main/genres_classification/genres_mod.parquet?raw=true'
cleaned_data_filename = 'processed_data.csv'

## If required - download environment packages

In [18]:
# !conda install pandas os pyarrow fastparquet

## Import libraries

In [19]:
import pandas as pd
import numpy as np
import os

## Prepare folders

In [20]:
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

## 1. Load data

In [21]:
df = pd.read_parquet(init_data_url)
df.sample(5)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,genre,song_name,title
4007,0.279,0.277,11,-17.168,0,0.0850,0.773000,0.9100,0.1380,0.0819,165.792,audio_features,259765,4,Dark Trap,Kenopsia,None
17352,0.735,0.482,8,-9.445,1,0.1390,0.006230,0.0144,0.4480,0.3150,181.995,audio_features,217933,4,RnB,In My Feelings,None
36462,0.560,0.943,1,-4.606,0,0.0800,0.001810,0.1510,0.0975,0.0382,174.034,audio_features,309306,4,dnb,None,Liquid Drum & Bass
39486,0.228,0.843,11,-5.711,0,0.0818,0.297000,0.4240,0.0958,0.0423,149.692,audio_features,233600,4,hardstyle,None,Hardstyle by Q-dance
24223,0.805,0.943,11,-6.503,0,0.0563,0.000749,0.8680,0.6790,0.8560,126.000,audio_features,249545,4,techhouse,None,Dirtybird//Techhouse by Mack\


## Explore the data

In [22]:
print('''
Data has {} columns and {} rows\n\n 
Data types:\n{}\n\n
Data summary:\n{}'''
.format(df.columns.values, df.shape[0], df.dtypes, df.describe()))


Data has ['danceability' 'energy' 'key' 'loudness' 'mode' 'speechiness'
 'acousticness' 'instrumentalness' 'liveness' 'valence' 'tempo' 'type'
 'duration_ms' 'time_signature' 'genre' 'song_name' 'title'] columns and 42896 rows

 
Data types:
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
type                 object
duration_ms           int64
time_signature        int64
genre                object
song_name            object
title                object
dtype: object


Data summary:
       danceability        energy           key      loudness          mode  \
count  42896.000000  42896.000000  42896.000000  33726.000000  42896.000000   
mean       0.639336      0.762560      5.368566     -6.463899      0.549492   
std        0.1

In [23]:
df.isnull().sum()
# We can ommit empty song name and titles and fill this with empty values as they are not a part of our analysis.
# It does not provide any value to our case.

danceability            0
energy                  0
key                     0
loudness             9170
mode                    0
speechiness             0
acousticness            0
instrumentalness        0
liveness                0
valence                 0
tempo                   0
type                    0
duration_ms             0
time_signature          0
genre                   0
song_name           21085
title               21817
dtype: int64

## Data cleansing

In [24]:
df_clean = df.copy()
print('Number of rows before cleansing: {}'.format(df_clean.size))

Number of rows before cleansing: 729232


### Deduplication

In [25]:
print('Data size before deduplication: {}'.format(df_clean.shape[0]))
df_clean = df_clean.drop_duplicates().reset_index(drop=True)
print('Data size after deduplication: {}'.format(df_clean.shape[0]))

Data size before deduplication: 42896
Data size after deduplication: 41975


### Handle NaN values

In [26]:
print('Number of NaN titles before replacement: {}'.format(df_clean['title'].isnull().sum()))
df_clean['title'].fillna(value='', inplace=True)
print('Number of NaN titles after replacement: {}'.format(df_clean['title'].isnull().sum()))

Number of NaN titles before replacement: 21240
Number of NaN titles after replacement: 0


In [27]:
print('Number of NaN song names before replacement: {}'.format(df_clean['song_name'].isnull().sum()))
df_clean['song_name'].fillna(value='', inplace=True)
print('Number of NaN song names after replacement: {}'.format(df_clean['song_name'].isnull().sum()))

Number of NaN song names before replacement: 20741
Number of NaN song names after replacement: 0


In [28]:
loudness_median = df_clean['loudness'].median()

print('Number of NaN loudness before replacement: {}'.format(df_clean['loudness'].isnull().sum()))
df_clean['loudness'].fillna(value=loudness_median, inplace=True)
print('Number of NaN loudness after replacement: {}'.format(df_clean['loudness'].isnull().sum()))

Number of NaN loudness before replacement: 9007
Number of NaN loudness after replacement: 0


In [29]:
print(df_clean.isnull().sum())
print(df_clean.shape)

danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
type                0
duration_ms         0
time_signature      0
genre               0
song_name           0
title               0
dtype: int64
(41975, 17)


## Feature engineering

In [30]:
df_clean['text_feature'] = df_clean['title'] + ' ' + df_clean['song_name']

## Save cleaned data

### Compare data dimensionality

In [31]:
print('Data before: {}\nData after: {}'.format(df.shape, df_clean.shape))

Data before: (42896, 17)
Data after: (41975, 18)


In [32]:
data_file = os.path.join(data_directory, cleaned_data_filename)
df_clean.to_csv(data_file, index=False)

In [33]:
df_clean.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,41975.000000,41975.000000,41975.000000,41975.000000,41975.000000,41975.000000,41975.000000,41975.000000,41975.000000,41975.000000,41975.00000,41975.000000,41975.000000
mean,0.639380,0.762855,5.371674,-6.418807,0.549184,0.136440,0.095894,0.284569,0.214190,0.356520,147.42499,251211.483240,3.972555
std,0.156495,0.183793,3.666697,2.609506,0.497581,0.126124,0.170647,0.371246,0.175699,0.233086,23.79437,103140.264753,0.268073
min,0.065100,0.000243,0.000000,-32.929000,0.000000,0.022700,0.000001,0.000000,0.010700,0.018700,57.96700,25600.000000,1.000000
25%,0.524000,0.632000,1.000000,-7.559000,0.000000,0.049100,0.001720,0.000000,0.099600,0.161000,129.91550,179998.000000,4.000000
50%,0.646000,0.804000,6.000000,-6.235000,1.000000,0.075400,0.016300,0.006450,0.135000,0.321000,144.96200,224951.000000,4.000000
75%,0.766000,0.923000,9.000000,-5.045500,1.000000,0.193000,0.106000,0.725000,0.294500,0.521000,160.97600,302013.000000,4.000000
max,0.988000,1.000000,11.000000,3.148000,1.000000,0.946000,0.988000,0.989000,0.988000,0.988000,220.29000,913052.000000,5.000000
